In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [6]:
from blackbox_wrapper import BlackboxWrapper
import keras
from datasets import build_synth
from variational_autoencoder import load_model
from neighborhood_generators import NeighborhoodGenerator
from lore_wrapper import LoreWrapper
from utils import choose_z
from joblib import load
from lasts import load_multiple_lasts
import time
import numpy as np
from experiments_neighborhoods_surrogates import multiple_tests
from ts_generator import PatternClassifier, TimeSeriesGenerator, LocalPattern, PolynomialClassifier, GlobalFeature
from sbgdt import Sbgdt, generate_n_shapelets_per_size
from saxdt import Saxdt

In [7]:
random_state = 0
np.random.seed(random_state)
dataset_name = "synth"
blackbox_name = "pat"

file_path = parentdir + "/saved/" + dataset_name + "_" + blackbox_name + "_" + time.strftime("%Y%m%d_%H%M%S")

(X_train, y_train, X_val, y_val,
 X_test, y_test, X_exp_train, y_exp_train,
 X_exp_val, y_exp_val, X_exp_test, y_exp_test) = build_synth(path=parentdir+"/datasets/synth04/",
                                                             random_state=random_state)

pat = load(parentdir + "/trained_models/synth04/synth04_pat.joblib")

_, _, autoencoder = load_model(parentdir + "/trained_models/synth04/synth04_vae")

blackbox = pat
encoder = autoencoder.layers[2]
decoder = autoencoder.layers[3]

DATASET INFO:
X SHAPE:  (600, 96, 1)
y SHAPE:  (600,)

CLASSES BALANCE
0 :  0.5
1 :  0.5

SHAPES:
BLACKBOX TRAINING SET:  (268, 96, 1)
BLACKBOX VALIDATION SET:  (68, 96, 1)
BLACKBOX TEST SET:  (84, 96, 1)
EXPLANATION TRAINING SET:  (115, 96, 1)
EXPLANATION VALIDATION SET:  (29, 96, 1)
EXPLANATION TEST SET:  (36, 96, 1)
Instructions for updating:
Colocations handled automatically by placer.


In [9]:
neighborhood_names = ["matched_uniform"]
neighborhood_list = [NeighborhoodGenerator]
neighborhood_kwargs_list = [
        {
            "n": 500,
            "n_search": 10000,
            "threshold": 2,
            "kind": "gaussian_matched",
            "sampling_kind": "uniform_sphere",
            "verbose": False,
            "redo_search": True,
            "forced_balance_ratio": 0.5,
            "cut_radius": True
        }
    ]

In [10]:
surrogate_kwargs_list = [{"random_state": np.random.seed(0), "create_plotting_dictionaries":False}]
surrogate_list = [Saxdt]
surrogate_names = ["saxdt"]

In [11]:
%%time
out = multiple_tests(
        X_exp_test,
        blackbox,
        encoder,
        decoder,
        neighborhood_list=neighborhood_list,
        neighborhood_kwargs_list=neighborhood_kwargs_list,
        neighborhood_names=neighborhood_names,
        surrogate_list=surrogate_list,
        surrogate_kwargs_list=surrogate_kwargs_list,
        surrogate_names=surrogate_names,
        file_path=file_path,
        random_state=random_state,
        n_instances_to_explain=None,
        verbose=True,
        simple_dump=[True],
        boxplots=True
    )

--- MULTIPLE LASTS TESTS ---

 --> RECONSTRUCTION ACCURACY: 0.9722222222222222


KeyboardInterrupt: 